# Modèle pour la détermination du type de véhicule

### 1. Préparation des données

Importation des datasets

In [ ]:
import os

#x_train = #TODO
y_train = [category.split('_')[0] for category in os.listdir('./result')]

print(y_train[:10])

['truck', 'cars', 'cars', 'cars', 'cars', 'truck', 'truck', 'truck', 'truck', 'cars']


Transformation : Niveau de gris sur tout le dataset

In [ ]:
import cv2, os

output_folder = './gray_images'

for image_name in os.listdir('./result') :
    img = cv2.imread(f'./result/{image_name}')
    grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, grayFrame)


Transformation : Floutage des images de meilleure qualité

Transformation : UpSampling en retournant les images aléatoirement

In [13]:
import tensorflow as tf
import numpy as np

def random_rotate_image(opencv_img):
    img = tf.convert_to_tensor(opencv_img, dtype=tf.float32)
    img = tf.image.rot90(img)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)

    img_rotated = img.numpy().astype(np.uint8)
    return img_rotated

output_folder = './rotated_gray_images'
os.makedirs(output_folder, exist_ok=True)

for image_name in os.listdir('./gray_images') :
    img = cv2.imread(f'./gray_images/{image_name}')

    rotations_list = [random_rotate_image(img), random_rotate_image(img)]
    for i, rotated_image in enumerate(rotations_list) :
        output_image_name = f"{os.path.splitext(image_name)[0]}_{i}.png"
        output_path = os.path.join(output_folder, output_image_name)
        cv2.imwrite(output_path, rotated_image)

Création du modèle